In [1]:
import tensorflow as tf
import json
import numpy as np
from sklearn.model_selection import train_test_split

2023-06-06 14:40:44.233985: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 14:40:44.295706: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 14:40:44.296568: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 14:40:45.466057: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file_path = '../dataset.json'

# Open the file and load its contents
with open(file_path, 'r') as json_file:
    # Read the file content
    file_content = json_file.read()

    # Split the content by newline to get individual JSON objects
    json_objects = file_content.split('\n')

data = []
# Process each JSON object
for json_object in json_objects:
    if json_object.strip() == '':
        continue  # Skip empty lines
    
    # Parse the JSON object
    data.append(json.loads(json_object))

print(len(data))

X = []
Y = []
Y_flat = []

for i in data:
    X.append(np.array(i['vector']))    
    Y.append(np.array(i['matrix']))
    
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
Y_train_flat =[i.flatten() for i in Y_train]
Y_test_flat =[i.flatten() for i in Y_test]

1331


# Handmade NN

In [3]:
# Define the neural network architecture
def build_crazy_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(output_dim)
    ])
    return model
def build_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(196)  # Output layer with shape (196,)
    ])
    return model

def build_model_recursive(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Reshape((1, 256)),  # Reshape input to match LSTM requirements
        # tf.keras.layers.LSTM(512, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='relu')),
        tf.keras.layers.LSTM(512),
        tf.keras.layers.Dense(output_dim)
    ])
    return model

# def build_model_convolutional(input_dim):
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
#         tf.keras.layers.Dense(7*7*128, activation='relu'),
#         tf.keras.layers.Reshape((7, 7, 128)),
#         tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2DTranspose(2, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2D(1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Reshape((14, 14, 1))  # Update the output shape to (14, 14, 1)
#     ])
#     return model

In [4]:
# Define the input and output dimensions
input_dim = 128  # Dimensionality of the input vector
output_dim = 196  # Dimensionality of the output vector

In [5]:
# # Build the model
# model = build_model_recursive(input_dim, output_dim)
# model_ann = build_model(input_dim, output_dim)
# model_conv = build_model_convolutional(input_dim)

# # Compile the model
# model.compile(optimizer='adam', loss='mse')
# model_ann.compile(optimizer='adam', loss='mse')
# model_conv.compile(optimizer='adam', loss='mse')

In [6]:
# model_ann.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=800)

In [7]:
# model.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=100)

In [8]:
# from sklearn.metrics import mean_squared_error as mse
# Y_predicted = model.predict(np.array(X_test))
# Y_ann_predicted = model_ann.predict(np.array(X_test))
# Y_conv_predicted = model_conv.predict(np.array(X_test))
# print(mse(Y_predicted, Y_test_flat))
# print(mse(Y_ann_predicted, Y_test_flat))

In [9]:
# Y_predicted = [np.reshape(i, (14,14)) for i in Y_predicted]
# Y_predicted

In [10]:
# Y_ann_predicted = [np.reshape(i, (14,14)) for i in Y_ann_predicted]
# Y_ann_predicted

In [11]:
# print(Y_conv_predicted)

# AutoKeras

In [12]:
import autokeras as ak
import numpy as np

In [13]:
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
train_set = tf.data.Dataset.from_tensor_slices((X_train, Y_train_flat))
test_set = tf.data.Dataset.from_tensor_slices((X_test, Y_test_flat))


In [ ]:

input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1000
)
reg.fit(train_set,epochs=11, validation_split=0.15)
predicted_y = reg.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)|
print(reg.evaluate(test_set))


In [ ]:
regressor = ak.StructuredDataRegressor(max_trials=100, overwrite=True)
regressor.fit(train_set, epochs=300, validation_split=0.15)
predicted_y = regressor.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
print(regressor.evaluate(test_set))

In [ ]:
model = reg.export_model()
model.summary()

In [ ]:
model_regressor = regressor.export_model()
model_regressor.summary()

In [ ]:
X_test = X_test.reshape(-1, 128)
Y_ak_predicted =reg.predict(X_test)

# Autogoal